# Neural Machine Translation Example

In [1]:
# Install TensorFlow and also our package via PyPI
!pip install tensorflow-gpu==2.0.0
!pip install headliner

     |████████████████████████████████| 380.8MB 44kB/s 
     |████████████████████████████████| 3.8MB 38.6MB/s 
     |████████████████████████████████| 450kB 59.7MB/s 
     |████████████████████████████████| 81kB 12.5MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found exis

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 450kB 14.0MB/s 
     |████████████████████████████████| 10.4MB 40.5MB/s 
     |████████████████████████████████| 1.0MB 62.2MB/s 
     |████████████████████████████████| 870kB 67.3MB/s 
     |████████████████████████████████| 2.2MB 46.1MB/s 
     |████████████████████████████████| 3.7MB 52.0MB/s 
     |████████████████████████████████| 122kB 66.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=2ff1e7f3cfa5d71db0c0556e70818e61c54d12c11a55cf18696ea0655d10bfb6
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing ins

In [3]:
# Download the German-English sentence pairs
!wget http://www.manythings.org/anki/deu-eng.zip
!unzip deu-eng.zip
!head deu.txt

--2020-01-24 10:05:02--  http://www.manythings.org/anki/deu-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:3037::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7747747 (7.4M) [application/zip]
Saving to: ‘deu-eng.zip’

deu-eng.zip         100%[===================>]   7.39M  9.36MB/s    in 0.8s    

2020-01-24 10:05:03 (9.36 MB/s) - ‘deu-eng.zip’ saved [7747747/7747747]

Archive:  deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              
Hi.	Hallo!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)
Hi.	Grüß Gott!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)
Run!	Lauf!	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #941078 (Fingerhut)
Wow!	Potzdonner!	CC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #

In [4]:
# Create the dataset but only take a subset for faster training
import io

def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[w for w in l.split('\t')]  for l in lines[:num_examples]]
    return zip(*word_pairs)

eng, ger, meta = create_dataset('deu.txt', 30000)
data = list(zip(eng, ger))
data[:5]

[('Hi.', 'Hallo!'),
 ('Hi.', 'Grüß Gott!'),
 ('Run!', 'Lauf!'),
 ('Wow!', 'Potzdonner!'),
 ('Wow!', 'Donnerwetter!')]

In [0]:
# Split the dataset into train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=100)

In [0]:
# Define the model and train it
import tensorflow as tf
from headliner.trainer import Trainer
from headliner.model.attention_summarizer import AttentionSummarizer

summarizer = AttentionSummarizer(lstm_size=1024, embedding_size=64, max_prediction_len=10)
trainer = Trainer(batch_size=64, 
                  steps_per_epoch=500, 
                  steps_to_log=50, 
                  max_output_len=10,
                  model_save_path='/tmp/summarizer')
trainer.train(summarizer, train, num_epochs=10, val_data=test)

training a bare model, preprocessing data to init model...
fitting tokenizers...
vocab encoder: 4675, vocab decoder: 7459
epoch 0, batch 50, logs: {'loss': 3.516734313964844}
epoch 0, batch 100, logs: {'loss': 3.1651382446289062}
epoch 0, batch 150, logs: {'loss': 2.9864306640625}
epoch 0, batch 200, logs: {'loss': 2.84840576171875}
epoch 0, batch 250, logs: {'loss': 2.73945068359375}
epoch 0, batch 300, logs: {'loss': 2.651563517252604}
epoch 0, batch 350, logs: {'loss': 2.5786356026785713}
epoch 0, batch 400, logs: {'loss': 2.5185585021972656}
epoch 0, batch 450, logs: {'loss': 2.462959255642361}
finished iterating over dataset, total batches: 467
epoch 0, batch 500, logs: {'loss': 2.407764404296875}

(input) <start> we've found it . <end> 
(target) <start> wir haben es gefunden . <end> 
(prediction) du kannst sie nicht . <end>


(input) <start> was tom caught ? <end> 
(target) <start> wurde tom erwischt ? <end> 
(prediction) hat tom tom ? <end>


(input) <start> whose baby is this ?

In [0]:
# Do some prediction
summarizer.predict('How are you?')

'wie geht es dir ? <end>'